In [4]:
import torch
import math
from transformer_lens import HookedTransformer

In [5]:
def normalize(R):
    with torch.no_grad():
        mean = R.mean(dim=0)
        R = R - mean
        norms = torch.norm(R, p=2, dim=1, keepdim=True)
        R = R/norms
    return R

def cal_cov(R):
    with torch.no_grad():
        Z = torch.nn.functional.normalize(R, dim=1)
        A = torch.matmul(Z.T, Z)/Z.shape[0]
    return A

def cal_entropy(A):
    with torch.no_grad():
        eig_val = torch.svd(A / torch.trace(A))[1] 
        entropy = - (eig_val * torch.log(eig_val)).nansum().item()
        normalized_entropy = entropy/math.log(A.shape[0])
    return normalized_entropy


In [6]:
prompt = "good job"

In [7]:
MODEL: HookedTransformer = HookedTransformer.from_pretrained("gpt2")
TOKENIZER = MODEL.tokenizer

input_ids = TOKENIZER.encode(prompt)

Loaded pretrained model gpt2 into HookedTransformer


In [8]:
d_model: int = MODEL.cfg.d_model
n_layers: int = MODEL.cfg.n_layers
n_heads: int = MODEL.cfg.n_heads
d_vocab: int = MODEL.cfg.d_vocab
print(f"{d_model=}, {n_layers=}, {n_heads=}, {d_vocab=}")

d_model=768, n_layers=12, n_heads=12, d_vocab=50257


In [9]:
input_ids

[11274, 1693]

In [13]:
MODEL(torch.tensor(input_ids))[0][:].shape

torch.Size([2, 50257])

In [27]:
MODEL(prompt).shape

torch.Size([1, 3, 50257])

In [11]:
with torch.no_grad():
   R = MODEL(torch.tensor(input_ids))[0][:]
   R = normalize(R)
   A = cal_cov(R)
   Entropy = cal_entropy(A) 
    
print(Entropy)

RuntimeError: Invalid buffer size: 9.41 GB